In [1]:
import json
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

data_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\search_20241106-223705_sodium+ion+battery+anode-sodium+ion+battery+cathode-sodium+ion+battery+electrode_annotated_50irrelevant_rephrased3.json'
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

texts = []
labels_raw = []

for i in data:
    texts.append(i["text"])
    labels_raw.append(i["label_int"])

labels = []
for i in labels_raw:
    if i == 0:
        labels.append(1)
    elif i == 1:
        labels.append(0)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42
)

train_dataset = Dataset.from_dict({
    'text': train_texts,
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'text': val_texts,
    'labels': val_labels
})

raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})
print(len(texts))

600


In [2]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 480
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 120
    })
})


In [3]:
model_name = "batterydata/batterybert-cased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, torch_dtype="auto")

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [6]:
import evaluate

metric = evaluate.load("accuracy")

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  eval_strategy="epoch",
                                  num_train_epochs=10)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)


In [10]:
trainer.train()

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.3411358892917633, 'eval_accuracy': 0.8833333333333333, 'eval_runtime': 0.9625, 'eval_samples_per_second': 124.681, 'eval_steps_per_second': 15.585, 'epoch': 1.0}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.12239693850278854, 'eval_accuracy': 0.975, 'eval_runtime': 1.0173, 'eval_samples_per_second': 117.964, 'eval_steps_per_second': 14.745, 'epoch': 2.0}


...

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 3.581124110496603e-05, 'eval_accuracy': 1.0, 'eval_runtime': 1.2715, 'eval_samples_per_second': 94.378, 'eval_steps_per_second': 11.797, 'epoch': 9.0}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 3.508112786221318e-05, 'eval_accuracy': 1.0, 'eval_runtime': 1.0549, 'eval_samples_per_second': 113.75, 'eval_steps_per_second': 14.219, 'epoch': 10.0}
{'train_runtime': 144.3961, 'train_samples_per_second': 33.242, 'train_steps_per_second': 4.155, 'train_loss': 0.10353800328448415, 'epoch': 10.0}


TrainOutput(global_step=600, training_loss=0.10353800328448415, metrics={'train_runtime': 144.3961, 'train_samples_per_second': 33.242, 'train_steps_per_second': 4.155, 'total_flos': 1262933065728000.0, 'train_loss': 0.10353800328448415, 'epoch': 10.0})

In [11]:
trainer.save_model("batterybert-cased-abstract_finetuned_600")

In [12]:
from transformers import pipeline
classifier = pipeline("text-classification", model="batterybert-cased-abstract_finetuned_600", tokenizer=tokenizer, device=0)

In [13]:
val_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai_1.json'
with open(val_path, 'r', encoding='utf-8') as f:
    val_data = json.load(f)

label_bert = []
score_bert = []
for i in range(len(val_data["text"])):
    val_label = classifier(val_data["text"][i][:512])
    if val_label[0]["label"] == "battery":
        label_bert.append(1)
        score_bert.append(val_label[0]["score"])
    elif val_label[0]["label"] == "non-battery":
        label_bert.append(0)
        score_bert.append(val_label[0]["score"])

val_data["label_bert"] = label_bert
val_data["score_bert"] = score_bert

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [14]:
label_openai = val_data["label_openai"]
label_annotated = val_data["label_annotated"]
print(f"No\tAnno\tOpenAI\tBERT\tBERT Score")

openai_tp = 0
openai_tn = 0
openai_fp = 0
openai_fn = 0
bert_tp = 0
bert_tn = 0
bert_fp = 0
bert_fn = 0

for i in range(len(label_bert)):
    if (label_bert[i] + label_openai[i] + label_annotated[i] != 3) and (label_bert[i] + label_openai[i] + label_annotated[i] != 0):
        print(f"{i}\t{label_annotated[i]}\t{label_openai[i]}\t{label_bert[i]}\t{score_bert[i]}")
        if label_bert[i] == 1 and label_annotated[i] == 0:
            bert_fp += 1
        elif label_bert[i] == 0 and label_annotated[i] == 1:
            bert_fn += 1
        if label_openai[i] == 1 and label_annotated[i] == 0:
            openai_fp += 1
        elif label_openai[i] == 0 and label_annotated[i] == 1:
            openai_fn += 1
    if label_bert[i] == 1 and label_annotated[i] == 1:
        bert_tp += 1
    if label_bert[i] == 0 and label_annotated[i] == 0:
        bert_tn += 1
    if label_openai[i] == 1 and label_annotated[i] == 1:
        openai_tp += 1
    if label_openai[i] == 0 and label_annotated[i] == 0:
        openai_tn += 1
        
print(f"OpenAI True Positive: {openai_tp}")
print(f"OpenAI True Negative: {openai_tn}")
print(f"OpenAI False Positive: {openai_fp}")
print(f"OpenAI False Negative: {openai_fn}")
print(f"Bert True Positive: {bert_tp}")
print(f"Bert True Negative: {bert_tn}")
print(f"Bert False Positive: {bert_fp}")
print(f"Bert False Negative: {bert_fn}")

No	Anno	OpenAI	BERT	BERT Score
7	0	1	1	0.7103307843208313
11	1	1	0	0.9999046325683594
12	1	1	0	0.852022647857666
19	1	1	0	0.9999538660049438
21	0	1	0	0.9999834299087524
27	1	1	0	0.9998410940170288
34	0	1	0	0.9999847412109375
36	0	1	0	0.999985933303833
39	1	1	0	0.9999593496322632
47	1	1	0	0.9999285936355591
53	0	1	0	0.9999818801879883
59	1	1	0	0.9990074038505554
79	0	1	0	0.9999827146530151
80	0	1	0	0.9999306201934814
82	1	1	0	0.9997178912162781
88	1	1	0	0.999763548374176
90	1	1	0	0.9302987456321716
92	1	1	0	0.999806821346283
97	1	1	0	0.9949029684066772
99	0	1	0	0.9966214895248413
OpenAI True Positive: 87
OpenAI True Negative: 5
OpenAI False Positive: 8
OpenAI False Negative: 0
Bert True Positive: 75
Bert True Negative: 12
Bert False Positive: 1
Bert False Negative: 12


In [15]:
val_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai.json'
with open(val_path, 'r', encoding='utf-8') as f:
    val_data = json.load(f)

label_bert = []
score_bert = []
for i in range(len(val_data["text"])):
    val_label = classifier(val_data["text"][i][:512])
    if val_label[0]["label"] == "battery":
        label_bert.append(1)
        score_bert.append(val_label[0]["score"])
    elif val_label[0]["label"] == "non-battery":
        label_bert.append(0)
        score_bert.append(val_label[0]["score"])

val_data["label_bert"] = label_bert
val_data["score_bert"] = score_bert

# result_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai_bert.json'
# with open(result_path, 'w', encoding='utf-8') as f:
#     json.dump(val_data, f, indent=4, ensure_ascii=False)



In [16]:
label_openai = val_data["label_openai"]
label_annotated = val_data["label_annotated"]
print(f"No\tAnno\tOpenAI\tBERT\tBERT Score")

openai_tp = 0
openai_tn = 0
openai_fp = 0
openai_fn = 0
bert_tp = 0
bert_tn = 0
bert_fp = 0
bert_fn = 0

for i in range(len(label_bert)):
    if (label_bert[i] + label_openai[i] + label_annotated[i] != 3) and (label_bert[i] + label_openai[i] + label_annotated[i] != 0):
        print(f"{i}\t{label_annotated[i]}\t{label_openai[i]}\t{label_bert[i]}\t{score_bert[i]}")
        if label_bert[i] == 1 and label_annotated[i] == 0:
            bert_fp += 1
        elif label_bert[i] == 0 and label_annotated[i] == 1:
            bert_fn += 1
        if label_openai[i] == 1 and label_annotated[i] == 0:
            openai_fp += 1
        elif label_openai[i] == 0 and label_annotated[i] == 1:
            openai_fn += 1
    if label_bert[i] == 1 and label_annotated[i] == 1:
        bert_tp += 1
    if label_bert[i] == 0 and label_annotated[i] == 0:
        bert_tn += 1
    if label_openai[i] == 1 and label_annotated[i] == 1:
        openai_tp += 1
    if label_openai[i] == 0 and label_annotated[i] == 0:
        openai_tn += 1
        
print(f"OpenAI True Positive: {openai_tp}")
print(f"OpenAI True Negative: {openai_tn}")
print(f"OpenAI False Positive: {openai_fp}")
print(f"OpenAI False Negative: {openai_fn}")
print(f"Bert True Positive: {bert_tp}")
print(f"Bert True Negative: {bert_tn}")
print(f"Bert False Positive: {bert_fp}")
print(f"Bert False Negative: {bert_fn}")

No	Anno	OpenAI	BERT	BERT Score
1	1	1	0	0.9949029684066772
2	1	1	0	0.999806821346283
4	0	1	1	0.9998987913131714
5	0	1	0	0.9999827146530151
8	1	0	1	0.9999698400497437
15	1	1	0	0.9999538660049438
22	1	0	0	0.999763548374176
29	1	1	0	0.9998410940170288
40	1	1	0	0.9302987456321716
51	1	1	0	0.9999285936355591
59	1	1	0	0.9997178912162781
63	0	1	0	0.9999818801879883
68	1	0	1	0.9999468326568604
80	1	1	0	0.9999046325683594
85	1	1	0	0.9990074038505554
90	0	0	1	0.7103307843208313
95	0	1	0	0.9999847412109375
103	0	1	0	0.999985933303833
112	0	1	1	0.9945092797279358
115	1	1	0	0.852022647857666
135	0	1	0	0.9966214895248413
136	0	1	1	0.9999493360519409
137	0	1	0	0.9999306201934814
139	1	1	0	0.9999593496322632
OpenAI True Positive: 84
OpenAI True Negative: 54
OpenAI False Positive: 9
OpenAI False Negative: 3
Bert True Positive: 75
Bert True Negative: 59
Bert False Positive: 4
Bert False Negative: 12
